# 入门

本笔记涵盖了如何开始使用聊天模型。接口是基于消息而不是纯文本的。

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [2]:
chat = ChatOpenAI(temperature=0)

您可以通过将一个或多个消息传递给聊天模型来获取聊天完成。响应将是一条消息。目前在LangChain中支持的消息类型有“AIMessage”、“HumanMessage”、“SystemMessage”和“ChatMessage”——“ChatMessage”需要传递一个任意的角色参数。大部分时间，您只需要处理“HumanMessage”、“AIMessage”和“SystemMessage”即可。

In [3]:
chat([HumanMessage(content="Translate this sentence from English to French. I love programming.")])

AIMessage(content="J'aime programmer.", additional_kwargs={})

OpenAI的聊天模型支持多个消息作为输入。有关更多信息，请参见[此处]（https://platform.openai.com/docs/guides/chat/chat-vs-completions）。以下是向聊天模型发送系统消息和用户消息的示例：

In [4]:
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming.")
]
chat(messages)

AIMessage(content="J'aime programmer.", additional_kwargs={})

您可以进一步生成多组消息的完成，使用“generate”。这将返回一个带有额外“message”参数的“LLMResult”。

In [5]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to French."),
        HumanMessage(content="I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(text="J'aime programmer.", generation_info=None, message=AIMessage(content="J'aime programmer.", additional_kwargs={}))], [ChatGeneration(text="J'aime l'intelligence artificielle.", generation_info=None, message=AIMessage(content="J'aime l'intelligence artificielle.", additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 57, 'completion_tokens': 20, 'total_tokens': 77}})

您可以从LLMResult中恢复诸如令牌使用情况之类的内容。

In [6]:
result.llm_output

{'token_usage': {'prompt_tokens': 57,
  'completion_tokens': 20,
  'total_tokens': 77}}

## PromptTemplates （提示模板）

您可以使用“MessagePromptTemplate”来利用模板。您可以从一个或多个“MessagePromptTemplates”构建一个“ChatPromptTemplate”。您可以使用“ChatPromptTemplate”的“format_prompt”——这将返回一个“PromptValue”，您可以将其转换为字符串或消息对象，具体取决于您是否希望将格式化值作为输入用于llm或chat模型。

为方便起见，模板上公开了“from_template”方法。如果您要使用此模板，则会像这样：

In [7]:
template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [8]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages())

AIMessage(content="J'adore la programmation.", additional_kwargs={})

如果您想更直接地构造“MessagePromptTemplate”，可以在外部创建一个提示模板，然后传递它，例如：

In [9]:
prompt=PromptTemplate(
    template="You are a helpful assistant that translates {input_language} to {output_language}.",
    input_variables=["input_language", "output_language"],
)
system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

## LLMChain （语言模型链）
您可以以与以前非常相似的方式使用现有的LLMChain-提供提示和模型。

In [10]:
chain = LLMChain(llm=chat, prompt=chat_prompt)

In [11]:
chain.run(input_language="English", output_language="French", text="I love programming.")

"J'adore la programmation."

## 流媒体

通过回调处理，支持“ChatOpenAI”的流式传输。

In [10]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat = ChatOpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)
resp = chat([HumanMessage(content="Write me a song about sparkling water.")])




Verse 1:
Bubbles rising to the top
A refreshing drink that never stops
Clear and crisp, it's pure delight
A taste that's sure to excite

Chorus:
Sparkling water, oh so fine
A drink that's always on my mind
With every sip, I feel alive
Sparkling water, you're my vibe

Verse 2:
No sugar, no calories, just pure bliss
A drink that's hard to resist
It's the perfect way to quench my thirst
A drink that always comes first

Chorus:
Sparkling water, oh so fine
A drink that's always on my mind
With every sip, I feel alive
Sparkling water, you're my vibe

Bridge:
From the mountains to the sea
Sparkling water, you're the key
To a healthy life, a happy soul
A drink that makes me feel whole

Chorus:
Sparkling water, oh so fine
A drink that's always on my mind
With every sip, I feel alive
Sparkling water, you're my vibe

Outro:
Sparkling water, you're the one
A drink that's always so much fun
I'll never let you go, my friend
Sparkling